In [2]:
import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

#%tensorflow_version 1.x
import tensorflow as tf
import cv2

In [3]:
train_data = np.load(r'FeatureFiles/train_imagedata.npy')
test_data = np.load(r'FeatureFiles/test_imagedata.npy')

In [4]:
class DeepLabModel(object):
    """Class to load deeplab model and run inference."""
    
    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    INPUT_SIZE = 513
    FROZEN_GRAPH_NAME = 'frozen_inference_graph'
    
    def __init__(self, tarball_path):
        """Creates and loads pretrained deeplab model."""
        self.graph = tf.Graph()

        graph_def = None
        # Extract frozen graph from tar archive.
        tar_file = tarfile.open(tarball_path)
        for tar_info in tar_file.getmembers():
            if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
                file_handle = tar_file.extractfile(tar_info)
                #graph_def = tf.GraphDef.FromString(file_handle.read())
                #graph = tf.Graph()
                graph_def = tf.compat.v1.GraphDef.FromString(file_handle.read())
                break

        tar_file.close()
        
        if graph_def is None:
            raise RuntimeError('Cannot find inference graph in tar archive.')
            
        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')

        self.sess = tf.compat.v1.Session(graph=self.graph)
        
    def run(self, image):
        """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
        #width, height = image.shape[0], image.shape[1]
        width, height = image.size
        resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
        target_size = (int(resize_ratio * width), int(resize_ratio * height))
        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
        #resized_image = np.resize(image, target_size)
        batch_seg_map = self.sess.run(
            self.OUTPUT_TENSOR_NAME,
            feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
        seg_map = batch_seg_map[0]
        
        return resized_image, seg_map
    
def create_ade20k_label_colormap():
        
    """Creates a label colormap used in ADE20K segmentation benchmark.

  Returns:
    A colormap for visualizing segmentation results.
  """
    colormap = np.asarray([
        [0,0,0],
        [120, 120, 120],
        [180, 120, 120],
        [6, 230, 230],
        [80, 50, 50],
        [4, 200, 3],
        [120, 120, 80],
        [140, 140, 140],
        [204, 5, 255],
        [230, 230, 230],
        [4, 250, 7],
        [224, 5, 255],
        [235, 255, 7],
        [150, 5, 61],
        [120, 120, 70],
        [8, 255, 51],
        [255, 6, 82],
        [143, 255, 140],
        [204, 255, 4],
        [255, 51, 7],
        [204, 70, 3],
        [0, 102, 200],
        [61, 230, 250],
        [255, 6, 51],
        [11, 102, 255],
        [255, 7, 71],
        [255, 9, 224],
        [9, 7, 230],
        [220, 220, 220],
        [255, 9, 92],
        [112, 9, 255],
        [8, 255, 214],
        [7, 255, 224],
        [255, 184, 6],
        [10, 255, 71],
        [255, 41, 10],
        [7, 255, 255],
        [224, 255, 8],
        [102, 8, 255],
        [255, 61, 6],
        [255, 194, 7],
        [255, 122, 8],
        [0, 255, 20],
        [255, 8, 41],
        [255, 5, 153],
        [6, 51, 255],
        [235, 12, 255],
        [160, 150, 20],
        [0, 163, 255],
        [140, 140, 140],
        [250, 10, 15],
        [20, 255, 0],
        [31, 255, 0],
        [255, 31, 0],
        [255, 224, 0],
        [153, 255, 0],
        [0, 0, 255],
        [255, 71, 0],
        [0, 235, 255],
        [0, 173, 255],
        [31, 0, 255],
        [11, 200, 200],
        [255, 82, 0],
        [0, 255, 245],
        [0, 61, 255],
        [0, 255, 112],
        [0, 255, 133],
        [255, 0, 0],
        [255, 163, 0],
        [255, 102, 0],
        [194, 255, 0],
        [0, 143, 255],
        [51, 255, 0],
        [0, 82, 255],
        [0, 255, 41],
        [0, 255, 173],
        [10, 0, 255],
        [173, 255, 0],
        [0, 255, 153],
        [255, 92, 0],
        [255, 0, 255],
        [255, 0, 245],
        [255, 0, 102],
        [255, 173, 0],
        [255, 0, 20],
        [255, 184, 184],
        [0, 31, 255],
        [0, 255, 61],
        [0, 71, 255],
        [255, 0, 204],
        [0, 255, 194],
        [0, 255, 82],
        [0, 10, 255],
        [0, 112, 255],
        [51, 0, 255],
        [0, 194, 255],
        [0, 122, 255],
        [0, 255, 163],
        [255, 153, 0],
        [0, 255, 10],
        [255, 112, 0],
        [143, 255, 0],
        [82, 0, 255],
        [163, 255, 0],
        [255, 235, 0],
        [8, 184, 170],
        [133, 0, 255],
        [0, 255, 92],
        [184, 0, 255],
        [255, 0, 31],
        [0, 184, 255],
        [0, 214, 255],
        [255, 0, 112],
        [92, 255, 0],
        [0, 224, 255],
        [112, 224, 255],
        [70, 184, 160],
        [163, 0, 255],
        [153, 0, 255],
        [71, 255, 0],
        [255, 0, 163],
        [255, 204, 0],
        [255, 0, 143],
        [0, 255, 235],
        [133, 255, 0],
        [255, 0, 235],
        [245, 0, 255],
        [255, 0, 122],
        [255, 245, 0],
        [10, 190, 212],
        [214, 255, 0],
        [0, 204, 255],
        [20, 0, 255],
        [255, 255, 0],
        [0, 153, 255],
        [0, 41, 255],
        [0, 255, 204],
        [41, 0, 255],
        [41, 255, 0],
        [173, 0, 255],
        [0, 245, 255],
        [71, 0, 255],
        [122, 0, 255],
        [0, 255, 184],
        [0, 92, 255],
        [184, 255, 0],
        [0, 133, 255],
        [255, 214, 0],
        [25, 194, 194],
        [102, 255, 0],
        [92, 0, 255],
    ])
    return colormap

def label_to_color_image(label):
        """Adds color defined by the dataset colormap to the label.

  Args:
    label: A 2D array with integer type, storing the segmentation label.

  Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the ADE20K color map.

  Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
  """
        if label.ndim != 2:
            raise ValueError('Expect 2-D input label')
        colormap = create_ade20k_label_colormap()
        
        if np.max(label) >= len(colormap):
            raise ValueError('label value too large.')
            
        return colormap[label]
    
def vis_segmentation(image, seg_map):
    """Visualizes input image, segmentation map and overlay view."""
    plt.figure(figsize=(15, 5))
    grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1]) #分为 1行4列,

    plt.subplot(grid_spec[0])  # 4个图的第一部分
    plt.imshow(image)
    plt.axis('off')
    plt.title('input image')

    plt.subplot(grid_spec[1])
    seg_image = label_to_color_image(seg_map).astype(np.uint8)
    plt.imshow(seg_image)
    plt.axis('off')
    plt.title('segmentation map')

    plt.subplot(grid_spec[2])
    plt.imshow(image)
    plt.imshow(seg_image, alpha=0.7)
    plt.axis('off')
    plt.title('segmentation overlay')

    unique_labels = np.unique(seg_map)
        
        
    ax = plt.subplot(grid_spec[3])
    plt.imshow(
        FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
    ax.yaxis.tick_right()
    plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
    plt.xticks([], [])
    ax.tick_params(width=0.0)
    plt.grid('off')
    plt.show()
        
    size = seg_image.shape[0] * seg_image.shape[1]
    selLABEL_NAMES = np.asarray([
                'tree', 'shrub', 'grass','road','sidewalk','sky','building'
                ])
    ratio_sum = ratio_plant = 0
    ratio_tree = ratio_shrub = ratio_grass = ratio_road = ratio_sidewalk = 0
        
    for m in seg_map.flatten():
        label = LABEL_NAMES[m]
        if label == 'tree':
            ratio_tree = ratio_tree+1
        if label == 'shrub':
            ratio_shrub = ratio_shrub+1
        if label == 'grass':
            ratio_grass = ratio_grass+1  
        if label == 'road':
            ratio_road = ratio_road+1 
        if label == 'sidewalk':
            ratio_sidewalk = ratio_sidewalk+1
                
        for n in selLABEL_NAMES:
            if label == n:
                ratio_sum = ratio_sum + 1
            
    ratio_tree_i = ratio_tree / len(seg_map.flatten())
    ratio_shrub_i = ratio_shrub / len(seg_map.flatten())
    ratio_grass_i = ratio_grass / len(seg_map.flatten())
    ratio_road_i = ratio_road / len(seg_map.flatten()) 
    ratio_sidewalk_i = ratio_sidewalk / len(seg_map.flatten()) 

    ratio_plant_i = (ratio_tree + ratio_shrub + ratio_grass)/ len(seg_map.flatten())
    #ratio_sum_i = ratio_sum / len(seg_map.flatten())
    print('tree合计占比为：',ratio_tree_i)
    print('shrub合计占比为：',ratio_shrub_i)
    print('grass合计占比为：',ratio_grass_i)
    print('road合计占比为：',ratio_road_i)
    print('sidewalk合计占比为：',ratio_sidewalk_i)
    print('plant经过计算植物合计占比为：',ratio_plant_i)
        
    return ratio_tree_i, ratio_shrub_i, ratio_grass_i, ratio_plant_i, ratio_road_i ,ratio_sidewalk_i

LABEL_NAMES = np.asarray([
        'ignore', 'wall', 'building', 'sky', 'floor', 'tree', 'ceiling', 'road', 
         'bed', 'window', 'grass', 'cabinet', 'sidewalk', 'person', 'ground', 'door', 
        'table', 'mount', 'shrub', 'curtain', 'chair', 'car', 'water', 'picture', 
        'couch', 'shelf', 'house', 'sea', 'mirror', 'rug', 'field', 'armchair',
        'seat', 'fence', 'desk', 'rock', 'clothes', 'lamp', 'bath', 'rail', 'cushion',
        'stand', 'box', 'pillar', 'signboard', 'drawers', 'counter', 'sand', 'sink',
        'skyscraper', 'fireplace', 'refrigerator', 'cupboard', 'path', 'steps',
        'runway', 'case', 'pool', 'pillow', 'screen', 'stairway', 'river', 
        'bridge', 'bookcase', 'blinds', 'coffeeTable', 'toilet', 'flower', 'book',
        'hill', 'bench', 'countertop', 'kitchen Sove', 'tree', 'kitchen', 
        'computingMachine', 'chair', 'boat', 'bar', 'machine', 'hut', 'bus', 
        'towel', 'light', 'truck', 'tower', 'chandelier', 'awning', 'streetlight',
        'booth', 'displayMonitor', 'airplane', 'dirtTrack', 'apparel', 'pole', 
        'ground', 'handrail', 'escalator', 'ottoman', 'bottle', 'counter', 'poster', 
        'stage', 'van', 'ship', 'fountain', 'conveyor', 'canopy', 'washer', 'toy', 
        'swimmingPool', 'stool', 'barrel', 'basket', 'waterfall', 'tent', 'bag', 
        'bike', 'cradle', 'oven', 'ball', 'food', 'step', 'container', 'brandLogo', 
        'oven', 'pot', 'animal', 'bicycle', 'lake', 'dishwasher', 'projectorScreen', 
        'blanket', 'statue', 'hood', 'sconce', 'vase', 'trafficLight', 'tray', 
        'GarbageBin', 'fan', 'dock', 'computerMonitor', 'plate', 'monitoringDevice', 
        'bulletinBoard', 'shower', 'radiator', 'drinkingGlass', 'clock', 'flag'
     ])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
#print('FULL_LABEL_MAP',FULL_LABEL_MAP)

FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)
#print('FULL_COLOR_MAP',FULL_COLOR_MAP)

In [5]:
MODEL_NAME = 'xception65_ade20k_train'  # @param ['mobilenetv2_ade20k_train', 'xception65_ade20k_train']

_DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
_MODEL_URLS = {
    'mobilenetv2_ade20k_train':
        'deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz',
    'xception65_ade20k_train':
        'deeplabv3_xception_ade20k_train_2018_05_29.tar.gz',
}
_TARBALL_NAME = 'deeplab_model.tar.gz'

model_dir = tempfile.mkdtemp()
tf.io.gfile.makedirs(model_dir)

download_path = os.path.join(model_dir, _TARBALL_NAME)
print('downloading model, this might take a while...')
urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME],
                   download_path)
print('download completed! loading DeepLab model...')

MODEL = DeepLabModel(download_path)
MODEL.INPUT_SIZE = 420

# Reduce image size if mobilenet model
if "mobilenetv2" in MODEL_NAME:
    MODEL.INPUT_SIZE = 257

print('model loaded successfully!')

downloading model, this might take a while...
download completed! loading DeepLab model...
model loaded successfully!


2022-07-26 14:20:01.419418: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 14:20:01.419548: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 14:20:01.461961: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/stud1/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-07-26 14:20:01.461972: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://w

In [6]:
def semantic_extractor(sample_count, data, model):
    semantic_features = np.zeros(shape=(sample_count, 151))
    img_num = 0
    
    while img_num < sample_count:
        img = data[img_num]*255
        img = img.astype(np.uint8)
        pil_img = Image.fromarray(img)
        
        resized_im, seg_map = model.run(pil_img)
        for i in range(len(seg_map)):
            for j in range(len(seg_map[i])):
                semantic_features[img_num][seg_map[i][j]] += 1
        if img_num % 100 == 0:
            print(f'Done for image {img_num}')
        img_num += 1
        
    return semantic_features

print('Started for train...')
train_semanticfeatures = semantic_extractor(18559, train_data, MODEL)
print('Done for train.')
print('Started for test...')
test_semanticfeatures = semantic_extractor(3277, test_data, MODEL)
print('Done for test')

Started for train...


2022-07-26 14:20:09.753104: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Done for image 0
Done for image 100
Done for image 200
Done for image 300
Done for image 400
Done for image 500
Done for image 600
Done for image 700
Done for image 800
Done for image 900
Done for image 1000
Done for image 1100
Done for image 1200
Done for image 1300
Done for image 1400
Done for image 1500
Done for image 1600
Done for image 1700
Done for image 1800
Done for image 1900
Done for image 2000
Done for image 2100
Done for image 2200
Done for image 2300
Done for image 2400
Done for image 2500
Done for image 2600
Done for image 2700
Done for image 2800
Done for image 2900
Done for image 3000
Done for image 3100
Done for image 3200
Done for image 3300
Done for image 3400
Done for image 3500
Done for image 3600
Done for image 3700
Done for image 3800
Done for image 3900
Done for image 4000
Done for image 4100
Done for image 4200
Done for image 4300
Done for image 4400
Done for image 4500
Done for image 4600
Done for image 4700
Done for image 4800
Done for image 4900
Done for ima

In [7]:
np.save(r'FeatureFiles/train_semanticfeatures.npy', train_semanticfeatures)
np.save(r'FeatureFiles/test_semanticfeatures.npy', test_semanticfeatures)